<a href="https://colab.research.google.com/github/SWE3T/TopicModeling/blob/main/SpaceNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Script that presents some text processing (NLP techniques)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import gensim
from gensim.models.phrases import Phrases, Phraser

import spacy
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')

import pandas as pd

import numpy as np

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

spacy.cli.download("en_core_web_md")
nlp = spacy.load('en_core_web_md')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [2]:
# function to clean a document: remove stopwords and punctuation
def clean(doc):
    stop_free = " ".join([w for w in doc.lower().split() if w not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    return punc_free

In [13]:
fpath='/content/drive/MyDrive/Códigos/Modelagem de tópicos/Final Work/dataset /'

data = pd.read_csv(fpath+'spacenews-december-2022.csv')
# docs=[]
# for d in in_f.readlines():
# 	docs.append(d[:-1])
# in_f.close()
# TDOC=1000 # total of documents to be tested
# NDOC=4 # running example document
# # keep the first TDOC documents for this  case study
# docs=docs[:TDOC]
# #document NDOC in the collection
# docs[NDOC]

In [8]:
data

,title,url,content,author,date,postexcerpt
0,Orion splashes down to end Artemis 1,https://spacenews.com/orion-splashes-down-to-e...,Updated at 5:45 p.m. Eastern after post-splash...,Jeff Foust,"December 11, 2022",Fifty years to the day after the last Apollo m...
1,Polaris Dawn crewed mission could suffer addit...,https://spacenews.com/polaris-dawn-crewed-miss...,LAS VEGAS — A billionaire-backed private astro...,Jeff Foust,"October 25, 2022",A billionaire-backed private astronaut mission...
2,DART on track for asteroid collision,https://spacenews.com/dart-on-track-for-astero...,WASHINGTON — A NASA spacecraft is on course to...,Jeff Foust,"September 25, 2022",A NASA spacecraft is on course to deliberately...
3,U.S. Space Command calls for investment in tec...,https://spacenews.com/u-s-space-command-calls-...,"WASHINGTON — Lt. Gen. John Shaw, deputy comman...",Sandra Erwin,"August 31, 2022",U.S. Space Command's Lt. Gen. John Shaw said '...
4,SpaceX requests permission for direct-to-smart...,https://spacenews.com/spacex-requests-permissi...,"TAMPA, Fla. — SpaceX could provide “full and c...",Jason Rainbow,"December 8, 2022",SpaceX could provide “full and continuous” dir...
...,...,...,...,...,...,...
18349,Kendall lays out Pentagon thinking on future s...,https://spacenews.com/frank-kendall-at-wsbr/,"\nFrank Kendall, the Pentagon’s top acquisitio...",SpaceNews Staff,"February 25, 2016","Frank Kendall, the Pentagon’s top acquisition ..."
18350,A larger share of NOAA’s declining space budge...,https://spacenews.com/a-larger-share-of-noaas-...,Updated Feb. 10 at 10:18 p.m. Eastern The U.S....,Debra Werner,"February 10, 2016",The U.S. National Oceanic and Atmospheric Admi...
18351,Think Tank Turns Its Attention To Mars As 2016...,https://spacenews.com/think-tank-turns-its-att...,WASHINGTON — As NASA develops a long-term stra...,Jeff Foust,"June 11, 2015",As NASA develops a long-term strategy to suppo...
18352,House Bill Leaves Last Three JPSS Satellites i...,https://spacenews.com/no-money-for-noaa-weathe...,WASHINGTON — A spending bill the House passed ...,Dan Leone,"June 4, 2015",A spending bill the House passed June 3 would ...


The dataset contains many unecessary data that won't be used for the topic modeling;

Fields such as **url**, **author**, and **date** will be removed.

It seems that the dataset, also includes an **postexcerpt**. Which seems to be an abstract of the text that is present in the content column. Although it could be used to make a biased guess on the topic of the text, I believe it's best to dont use it for now, and use only the content of the article.

In [14]:
data = data.drop(['url', 'date', 'author', 'postexcerpt'], axis=1)

In [15]:
data

,title,content
0,Orion splashes down to end Artemis 1,Updated at 5:45 p.m. Eastern after post-splash...
1,Polaris Dawn crewed mission could suffer addit...,LAS VEGAS — A billionaire-backed private astro...
2,DART on track for asteroid collision,WASHINGTON — A NASA spacecraft is on course to...
3,U.S. Space Command calls for investment in tec...,"WASHINGTON — Lt. Gen. John Shaw, deputy comman..."
4,SpaceX requests permission for direct-to-smart...,"TAMPA, Fla. — SpaceX could provide “full and c..."
...,...,...
18349,Kendall lays out Pentagon thinking on future s...,"\nFrank Kendall, the Pentagon’s top acquisitio..."
18350,A larger share of NOAA’s declining space budge...,Updated Feb. 10 at 10:18 p.m. Eastern The U.S....
18351,Think Tank Turns Its Attention To Mars As 2016...,WASHINGTON — As NASA develops a long-term stra...
18352,House Bill Leaves Last Three JPSS Satellites i...,WASHINGTON — A spending bill the House passed ...


In [ ]:
# clean the collection and store the restult into docs_tk
docs_tk = [clean(doc) for doc in docs]
#show the NDOC clean document
docs_tk[NDOC]

'synopsis  mentally unstable man undergoing psychotherapy saves boy potentially fatal accident falls love boy  mother  fledgling restauranteur  unsuccessfully attempting gain woman  favor  takes pictures kills number people way  comments  stalked yet another seemingly endless string spurned  psychos  getting   revenge type movies stable category 1990s film industry  theatrical direct   video  proliferation may due part fact  typically inexpensive produce  special effects  big name stars  serve vehicles flash nudity  allowing frequent late  night cable television   stalked wavers slightly norm one respect  psycho never actually affair  contrary   rejected rather quickly  psycho typically ex  lover  ex  wife  ex  husband    stalked another redundant entry doomed collect dust video shelves viewed midnight cable  stalked provide much suspense  though sets  interspersed throughout opening credits  instance  serious  sounding narrator spouts statistics stalkers ponders may cause man stalk   

In [ ]:
print(len(docs),'-',len(docs_tk))

1000 - 1000


In [ ]:
#tokenize the documents (and eliminate words with less than 3 letters)
docs_tk=[gensim.utils.simple_preprocess(d, deacc= True, min_len=3) for d in docs_tk]
len_token=[len(d) for d in docs_tk]
#some statistics
print('# of words:', sum(len_token))
print('Average # of words:', sum(len_token)/len(len_token))

# of words: 326218
Average # of words: 326.218


In [ ]:
# generate bigrams
# docs_bg is a copy from docs_tk with bigrams
phrases  = Phrases(docs_tk, min_count = 5,threshold=.9,scoring='npmi') #10 e 5 default threshold 20 (npmi from -1 to 1)
bigram=Phraser(phrases)
docs_bg=[bigram[d] for d in docs_tk]
#dictionary=Dictionary(docs_bg)
#dictionary.filter_extremes(keep_n = 10000, no_above= 0.8, no_below = len(docs_bg)*.1)
#corpus=[dictionary.doc2bow(d) for d in docs_bg]
#print(len(docs_bg))
docs_bg[NDOC]

#Just to show the built bigrams

In [ ]:
uniqw=set()
#show the built bigrams
for i,d in enumerate(docs_bg):
  for w in d:
    if '_' in w:
      uniqw.add(w)
      print(i,w)
print(len(uniqw))

## Let's test stemming, lemmatization (spacy and nltk)

In [ ]:
# docs_lm is a copy from docs_bg but lemmatized (using spacy)
docs_lm=[]
for i,d in enumerate(docs_bg):
  print(i,end='')
  d=" ".join([w for w in d])
  tdoc=nlp(d)
  lm=" ".join([token.lemma_ for token in tdoc]) # if not(token.is_stop == True or token.is_punct == True or '_' in token.lemma_)])
  docs_lm.append(lm)
  print('\r\r\r\r\r\r\r\r',end='')
print()
print(len(docs_lm))

999
1000


In [ ]:
x=nlp('cannot')
[token.lemma_ for token in x]

['can', 'not']

In [ ]:
#Part Of Speech (POS) using spacy
mydoc=nlp(' '.join([w for w in docs_bg[NDOC]]))
print(f"{'text':{15}} {'POS':{6}} {'TAG':{6}} {'Dep':{10}} {'POS explained':{25}} {'tag explained'} ")
print('='*100)
for token in mydoc: #visit all words in the NDOC document to check their POS
  print(f'{token.text:{15}} {token.pos_:{6}} {token.tag_:{6}} {token.dep_:{10}} {spacy.explain(token.pos_):{25}} {spacy.explain(token.tag_)}')


In [ ]:
# NDOC document with lemmatization
' '.join([w for w in docs_lm[NDOC].split()])

In [ ]:
docs_bg[NDOC]

In [ ]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer() #(using nltk)
# how the words from the first document: original, stemm, lemmatize (nltk), lemmatize (spacy)
print(f"{'Original':{15}} {'Stemmed':{15}} {'Lem NLTK':{15}} {'Lem SPACY':{15}}'")
print("="*60)
dlm=docs_lm[NDOC].split()
for i,w in enumerate(docs_bg[NDOC]):
  print(f'{w:{15}} {ps.stem(w):{15}} {lemmatizer.lemmatize(w):{15}} {dlm[i]:{15}}')